In [1]:
# import lib
import cv2
import os
import base64
import requests
import pickle
import numpy as np

In [2]:
url = "http://localhost:8080/api/gethog" 

#list
cars_brand = [
    "Audi",
    "Hyundai Creta",
    "Mahindra Scorpio",
    "Rolls Royce",
    "Swift",
    "Tata Safari",
    "Toyota Innova"
]

In [3]:
#bases64 
def img64HOG(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data," + str.split(str(img_str), "'")[1]
    response = requests.get(url, json={"img": data})
    return response.json()

In [4]:
#read data
def readDataImg(path):
    y = []
    for sub in os.listdir(path):
        for fn in os.listdir(path + '/' + sub):
            img_file_name = path + '/' + sub + "/" + fn
            img = cv2.imread(img_file_name)
            res = img64HOG(img)
            hog = list(res["hog"])
            hog.append(cars_brand.index(sub))
            y.append(hog)
    return y

In [5]:
#sava and load pkl
def savePkl(filename ,path):
    cars = readDataImg(path)  
    
    write_path = filename + ".pkl" 
    pickle.dump(cars, open(write_path,"wb"))
    print("data preparation is done")

def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [7]:
# เรียกใช้ ฟังก์ชัน savePlk เพื่อ save .pkl ส่งชื่อไฟล์และpath ของ  train 
train_dir = r'Cars Dataset\train'
savePkl('carsTrain',train_dir)

data preparation is done


In [8]:
# เรียกใช้ ฟังก์ชัน savePlk เพื่อ save .pkl ส่งชื่อไฟล์และpath ของ  test
test_dir = r'Cars Dataset\test'
savePkl('carsTest',test_dir)

data preparation is done


In [9]:
# เรียกใช้ ฟังก์ชัน loadPlk เพื่อ load  .pkl ส่งชื่อไฟล์
train_data = loadPkl('carsTrain')
print(len(train_data))
test_data = loadPkl('carsTest')
print(len(test_data))

3352
813


In [10]:
# list to array
train_np = np.array(train_data)
X_train = train_np[:,0:-1]
y_train = train_np[:,-1]

test_np = np.array(test_data)
X_test = test_np[:,0:-1]
y_test = test_np[:,-1]

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [12]:
clf = DecisionTreeClassifier() #สร้างต้นไม้
clf.fit(X_train,y_train) #เรียนรู้ข้อมูล

DecisionTreeClassifier()

In [13]:
#เรียนรู้ เพื่อทำนาย TEST และหาค่า Accuracy Score, Confusion Matrix
y_model_pred = clf.predict(X_test) 
print('Accuracy Score',accuracy_score(y_test,y_model_pred)*100)
print('Confusion Matrix : \n',confusion_matrix(y_test,y_model_pred))

Accuracy Score 46.98646986469865
Confusion Matrix : 
 [[ 70  16  18  26  21  21  27]
 [  8  41   5   1   0   2  10]
 [ 10   3  26  12   1  10  13]
 [ 26   4   8  13   4  10   9]
 [  7  11   5   5  51   1  22]
 [ 14   4   1   5   8  66   8]
 [ 26  11   6   6  17   9 115]]


In [14]:
#SAVE MODEL
path_model = 'carsbrandmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))